In [36]:
import requests
import json
import re
from pprint import pprint
import pandas as pd


def get_holdings(url,keys):
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0"
    }
    # Initialize an empty list to store the extracted data
    holdings_data = []
    with requests.Session() as req:
        req.headers.update(headers)
        for key in keys:
            r = req.get(url.format(key))
            for line in r.text.splitlines():
                if not line.startswith('etf_holdings.formatted_data'):
                    continue
                data = json.loads(line[30:-1])
                for holding in data:
                    goal = re.search(r'etf/([^"]*)', holding[1])
                    if goal:
                        # Append relevant data to the list
                        holdings_data.append([goal.group(1), *holding[2:5]])
                break
    return holdings_data

In [43]:
schd_holdings = get_holdings("https://www.zacks.com/funds/etf/{}/holding",['schd'])
columns = ['symbol', 'shares', 'weight', '52wk_change']
df_holdings = pd.DataFrame(schd_holdings, columns=columns)
df_holdings

,symbol,shares,weight,52wk_change
0,VZ,"57,891,834",4.54,4.67
1,AMGN,"7,946,436",4.38,17.64
2,AVGO,"2,268,487",4.29,113.30
3,KO,"33,663,538",4.02,-1.34
4,ABBV,"13,589,325",3.96,16.60
...,...,...,...,...
96,ETD,"369,919",0.02,12.57
97,EBF,"414,152",0.02,-7.52
98,HAFC,"488,760",0.02,-32.92
99,CPF,"439,861",0.02,-12.72


In [38]:
import numpy as np
vector_holdings = df_holdings.loc[:,['symbol','weight']]
vector_holdings['weight_trading212']='0.1'
vector_schd = np.array(vector_holdings['weight'].astype(float))
initial_guess = np.array(vector_holdings['weight_trading212'].astype(float))

In [15]:
rounded_vector = [round(element, 1) for element in vector_schd]
rounded_vector =  [0.1 if element == 0 else element for element in rounded_vector]

In [16]:
sum_of_rounded_vector = sum(rounded_vector)
rounded_sum = round(sum_of_rounded_vector)
number_of_reduction = (rounded_sum-100)/0.1

In [17]:
number_of_reduction = (rounded_sum-100)/0.1
i=len(rounded_vector)-1
while number_of_reduction > 0:
    if(rounded_vector[i]>0.1):
        rounded_vector[i]=rounded_vector[i]-0.1
        number_of_reduction= number_of_reduction-1
    i=i-1

In [18]:
rounded_vector = [round(element, 1) for element in rounded_vector]
vector_holdings['weight_trading212']=rounded_vector
vector_holdings['symbol']= vector_holdings['symbol']+'_US_EQ'
vector_holdings

,symbol,weight,weight_trading212
0,UNH_US_EQ,9.43,9.4
1,GS_US_EQ,6.77,6.8
2,MSFT_US_EQ,6.46,6.5
3,HD_US_EQ,6.21,6.2
4,MCD_US_EQ,5.12,5.1
5,CAT_US_EQ,5.03,5.0
6,AMGN_US_EQ,4.88,4.9
7,V_US_EQ,4.57,4.6
8,CRM_US_EQ,4.54,4.5
9,BA_US_EQ,4.52,4.5


In [19]:
from scipy.spatial import distance
euclidean_distance = distance.euclidean(vector_holdings['weight'].astype(float), vector_holdings['weight_trading212'].astype(float))
euclidean_distance

0.167332005306815

In [26]:
vector_holdings['weight_trading212'][0]=9.6

C:\Users\adamj\AppData\Local\Temp\ipykernel_380\507860111.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vector_holdings['weight_trading212'][0]=9.6


In [27]:
vector_holdings['weight_trading212']=vector_holdings['weight_trading212'].round(1)

In [28]:
vector_holdings['weight_trading212'].sum()

100.10000000000001

In [29]:
vector_holdings['weight_trading212']=vector_holdings['weight_trading212']/100

In [30]:
json_data = vector_holdings.set_index('symbol').to_dict()['weight_trading212']

In [31]:
sum(json_data.values())

1.0010000000000003

In [32]:
json_data

{'UNH_US_EQ': 0.096,
 'GS_US_EQ': 0.068,
 'MSFT_US_EQ': 0.065,
 'HD_US_EQ': 0.062,
 'MCD_US_EQ': 0.051,
 'CAT_US_EQ': 0.05,
 'AMGN_US_EQ': 0.049,
 'V_US_EQ': 0.046,
 'CRM_US_EQ': 0.045,
 'BA_US_EQ': 0.045,
 'HON_US_EQ': 0.036000000000000004,
 'AAPL_US_EQ': 0.035,
 'TRV_US_EQ': 0.032,
 'AXP_US_EQ': 0.032,
 'JPM_US_EQ': 0.028999999999999998,
 'IBM_US_EQ': 0.028999999999999998,
 'JNJ_US_EQ': 0.027999999999999997,
 'WMT_US_EQ': 0.027000000000000003,
 'CVX_US_EQ': 0.026000000000000002,
 'PG_US_EQ': 0.026000000000000002,
 'NKE_US_EQ': 0.021,
 'MMM_US_EQ': 0.019,
 'MRK_US_EQ': 0.019,
 'DIS_US_EQ': 0.017,
 'KO_US_EQ': 0.01,
 'DOW_US_EQ': 0.01,
 'CSCO_US_EQ': 0.009000000000000001,
 'INTC_US_EQ': 0.008,
 'VZ_US_EQ': 0.006999999999999999,
 'WBA_US_EQ': 0.004}

In [33]:
'''
with open('schd_holdings212.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=4)
'''

"\nwith open('schd_holdings212.json', 'w') as json_file:\n    json.dump(json_data, json_file, indent=4)\n"

In [35]:
json_data = vector_holdings.set_index('symbol').to_dict()['weight_trading212']
import requests

url = "https://live.trading212.com/api/v0/equity/pies"

payload = {
  "dividendCashAction": "REINVEST",
  "endDate": "2019-08-24T14:15:22Z",
  "goal": 0,
  "icon": "Home",
  "instrumentShares": json_data,
  "name": "dj30"
}

headers = {
  "Content-Type": "application/json",
  "Authorization": "yuour key"
}

response = requests.post(url, json=payload, headers=headers)

data = response.json()
print(data)

{'code': 'Instrument with ticker DOW_US_EQ cannot be found in our equity instruments'}
